In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

import couponProjectUtils as proj_utils

from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SMOTE

## models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

## to make it possible to display multiple output inside one cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.options.display.max_rows = 50
pd.set_option('display.float_format', lambda x: '%9.8f' % x)

# Use best encoding for coupon type: Restaurant(20-50)

In [2]:
## load data, split Xy, upsample with SMOTE, split train/test
data = pd.read_csv('data/Restaurant20To50_data_encoded-age_nominal.csv')

## define X and y 
X = data.drop('Y', axis=1).reset_index(drop=True)
y = data.Y

## Handle imbalance
smote = SMOTE(random_state=42)

orig_vc = y.value_counts()
print(f"Orig: {100* orig_vc[1]/(orig_vc.sum()):.2f}% Yes")

X_sm, y_sm = smote.fit_resample(X, y)

## Data splitting train/test
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.20, random_state=42)

print("After SMOTE upsampling")
train_vc = y_train.value_counts()
print(f"\tNum training samples: {X_train.shape[0]:,} ({100* train_vc[1]/(train_vc.sum()):.2f}% yes)")
test_vc = y_test.value_counts()
print(f"\tNum testing samples:  {X_test.shape[0]:,} ({100* test_vc[1]/(test_vc.sum()):.2f}% yes)")

Orig: 44.15% Yes
After SMOTE upsampling
	Num training samples: 1,329 (51.17% yes)
	Num testing samples:  333 (45.35% yes)


In [3]:
## Replace the unknown values (previously encoded as 0.0) using the most frequent value seen in the TRAINING data
X_train, X_test, train_modes = proj_utils.replace_unknown_with_mode(X_train, X_test, 
                                                                    outfile_prefix='phase2-Restaurant20To50')

## Testing RandomForestClassifier with Grid Search Cross-validation


In [4]:
cols_to_view = ['rank_test_score', 'param_max_features', 'param_min_samples_split', 'param_n_estimators', 
                'mean_test_score', 'std_test_score', 'mean_train_score', 'std_train_score']

param_grid={'max_features': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
            'min_samples_split': [2, 3, 4, 5],
            'n_estimators': [100, 150, 200]}

clf = RandomForestClassifier(oob_score=True, random_state=42)

grid_search1 = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=5, return_train_score=True, n_jobs=4)
grid_search1.fit(X_train.values, y_train.values)

cv_results1 = pd.DataFrame(grid_search1.cv_results_)
best_model = grid_search1.best_estimator_
best_model

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(oob_score=True, random_state=42),
             n_jobs=4,
             param_grid={'max_features': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                          13, 14, 15],
                         'min_samples_split': [2, 3, 4, 5],
                         'n_estimators': [100, 150, 200]},
             return_train_score=True, scoring='roc_auc')

RandomForestClassifier(max_features=4, n_estimators=150, oob_score=True,
                       random_state=42)

In [5]:
print(f"Best model oob score: {best_model.oob_score_}")
y_test_score = best_model.predict_proba(X_test)[:, 1]
print(f"ROC AUC: TEST {roc_auc_score(y_test, y_test_score):.6f}\n")

Best model oob score: 0.7027840481565086
ROC AUC: TEST 0.774452



In [6]:
cv_results1.loc[:, cols_to_view].sort_values('rank_test_score').head(10)

,rank_test_score,param_max_features,param_min_samples_split,param_n_estimators,mean_test_score,std_test_score,mean_train_score,std_train_score
25,1,4,2,150,0.77125342,0.01560944,0.99999327,0.00000518
13,2,3,2,150,0.77107348,0.00923833,0.99999327,0.00000518
26,3,4,2,200,0.77078221,0.01369191,0.99999327,0.00000518
14,4,3,2,200,0.77035975,0.01146327,0.99999327,0.00000518
17,5,3,3,200,0.77035721,0.01327603,0.99998265,0.00001011
12,6,3,2,100,0.76969777,0.00993328,0.99999327,0.00000518
20,7,3,4,200,0.76962994,0.01653365,0.99985094,0.00005155
16,8,3,3,150,0.76914431,0.01513016,0.99998124,0.00001054
65,9,7,3,200,0.76910024,0.01584481,0.99998903,0.00000467
52,10,6,3,150,0.76855164,0.01505991,0.99998549,0.00000518


In [23]:
featimp_df = pd.DataFrame({'feature_name': X.columns, 'importance': best_model.feature_importances_})
featimp_df.sort_values('importance', ascending=False).head(10)

,feature_name,importance
9,income,0.07564337
13,CoffeeHouse,0.06104137
16,Restaurant20To50,0.06088167
8,education,0.05403117
14,CarryAway,0.05053496
15,RestaurantLessThan20,0.05007116
12,Bar,0.04973657
7,time,0.04624255
24,expiration_2h,0.04099355
10,temperature,0.03400992


In [8]:
import pickle

with open('model/phase2/Restaurant20To50_GridSearchCV_RFC.pickle', "wb") as f:
    pickle.dump(grid_search1,f)
    
with open('model/phase2/Restaurant20To50_best_model_RFC.pickle', "wb") as f:
    pickle.dump(best_model,f)

In [18]:
## save train/test info to file
Xy_tt = {'X_train': X_train, 'X_test': X_test, 'y_train': y_train, 'y_test': y_test}

for k in Xy_tt.keys():
    fname = f'data/Restaurant20To50_{k}_phase2.csv'
    print(f'Saving file: {fname}')
    Xy_tt[k].to_csv(fname, index=False)


Saving file: data/Restaurant20To50_X_train_phase2.csv
Saving file: data/Restaurant20To50_X_test_phase2.csv
Saving file: data/Restaurant20To50_y_train_phase2.csv
Saving file: data/Restaurant20To50_y_test_phase2.csv


# KNN

In [9]:
param_grid={'n_neighbors': [x for x in range(1,12)]}

clf = KNeighborsClassifier()

grid_search_knn = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=5, return_train_score=True, n_jobs=4)
grid_search_knn.fit(X_train.values, y_train.values)

knn_cols_to_view = ['rank_test_score', 'param_n_neighbors', 
                    'mean_test_score', 'std_test_score', 'mean_train_score', 'std_train_score']

cv_results_knn = pd.DataFrame(grid_search_knn.cv_results_)
cv_results_knn.loc[:, knn_cols_to_view].sort_values('rank_test_score').head(5)

best_model_knn = grid_search_knn.best_estimator_
best_model
y_test_score_knn = best_model_knn.predict_proba(X_test)[:, 1]
print(f"ROC AUC: TEST {roc_auc_score(y_test, y_test_score_knn):.6f}\n")

GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=4,
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]},
             return_train_score=True, scoring='roc_auc')

,rank_test_score,param_n_neighbors,mean_test_score,std_test_score,mean_train_score,std_train_score
2,1,3,0.68208149,0.01919790,0.90766762,0.00471535
1,2,2,0.68165807,0.02287436,0.94713792,0.00350557
3,3,4,0.67914492,0.01290695,0.87633364,0.00317121
4,4,5,0.67863999,0.02253098,0.85305018,0.00566982
6,5,7,0.66734273,0.01973135,0.82170053,0.00496642


RandomForestClassifier(max_features=4, n_estimators=150, oob_score=True,
                       random_state=42)

ROC AUC: TEST 0.733134



In [10]:
with open('model/phase2/Restaurant20To50_GridSearchCV_KNN.pickle', "wb") as f:
    pickle.dump(grid_search_knn,f)
    
with open('model/phase2/Restaurant20To50_best_model_KNN.pickle', "wb") as f:
    pickle.dump(best_model_knn,f)

# Logistic regression

In [24]:
## standardization
std_scaler=StandardScaler().fit(X_train) ## first fit the model on the training data

## now std scaler transform
X_train_scaled = std_scaler.transform(X_train)
X_test_scaled = std_scaler.transform(X_test)

## apply regression
liblinear_model = LogisticRegression(solver='liblinear', random_state=42)
liblinear_model.fit(X_train_scaled, y_train)   # train model by fitting to training data

## use cv to assess generalization
cross_val_scores = cross_val_score(liblinear_model, X_train, y_train, scoring='roc_auc', cv=5)
print(f"5-fold CV ROC AUC: mean: {np.mean(cross_val_scores):.6f} std dev:{np.std(cross_val_scores):.6f}")
    
# use trained model to get predicted target var
y_test_score_ll = liblinear_model.predict_proba(X_test_scaled)[:, 1]

print(f"ROC AUC: TEST {roc_auc_score(y_test, y_test_score_ll)}\n")

## apply regression again, using a different solver
lbfgs_model = LogisticRegression(solver='lbfgs', random_state=42)
lbfgs_model.fit(X_train_scaled, y_train)   # train model by fitting to training data

## use cv to assess generalization
cross_val_scores = cross_val_score(lbfgs_model, X_train, y_train, scoring='roc_auc', cv=5)
print(f"5-fold CV ROC AUC: mean: {np.mean(cross_val_scores):.6f} std dev:{np.std(cross_val_scores):.6f}")

# use trained model to get predicted target var
y_test_score_lb = lbfgs_model.predict_proba(X_test_scaled)[:, 1]

print(f"ROC AUC: TEST {roc_auc_score(y_test, y_test_score_lb)}\n")

LogisticRegression(random_state=42, solver='liblinear')

5-fold CV ROC AUC: mean: 0.732574 std dev:0.008737
ROC AUC: TEST 0.7023870169565534



LogisticRegression(random_state=42)

5-fold CV ROC AUC: mean: 0.733931 std dev:0.009610
ROC AUC: TEST 0.7023870169565534



In [26]:
## both LR models have essentially the same score, just save one
with open('model/phase2/Restaurant20To50_best_model_LogisticRegression.pickle', "wb") as f:
    pickle.dump(liblinear_model,f)